### Imports

In [ ]:
import json
import numpy as np  # Algebra library
import tensorflow as tf  # Tensorflow
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\3021599292.py", line 3, in <module>
    import tensorflow as tf  # Tensorflow
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\tensorflow\__init__.py", line 102, in <module>
    from tensorflow_core import *
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\tensorflow_core\__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "<frozen importlib._bootstrap>", line 1019, in _handle_fromlist
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "C:\Us

## Define Parameters
Specify the path of the test data and the parameters used for training to load the correct model

### data.json format:
{
    "input_size": [
        X,
        Y,
        Channels
    ],
    "num_classes": Number Of Classes,
    "class_names": [
        "Class 1",
        "Class 2"
    ]
}



In [ ]:
# Hardcoded values for training parameters
# Path to .5 file for test data
test_data = "data\\CatOrDog\\test.h5"

# Set this to none if you will manually input the needed variables
dataJson = "data\\CatOrDog\\data.json"

checkpoint_path = r"C:\Users\Michael\Desktop\Model Benchmark\tensorboard\Models\CatOrDogStandard16_lr0.0001/my_capsule_network"

# Folder Name Of The Saved Model
folder_name = "CatOrDogStandard"

# Image dimensions that the model was trained on
# Manually Set This If data.json does not exist, otherwise can leave as None
# input_dimensions = [96, 96, 1]
input_dimensions = None

# Class Names
# Manually Set This If data.json does not exist, otherwise can leave as None
# class_names = ["Cat", "Dog"]
class_names = None

# Number of classes
# Manually Set This If data.json does not exist, otherwise can leave as None
# num_classes = 2
num_classes = None

# Automatically read the input_dimensions, class_names and num_classes from a data.json if it exists
if dataJson is not None:
    with open(dataJson, "r") as f:
        data = json.load(f)

    input_dimensions = data['input_size']
    class_names = data['class_names']
    num_classes = data['num_classes']

    f.close()

# # Batch Size The Model Was Trained With
# batch_size = 16
# 
# # How Many Epochs The Model Was Trained With
# # n_epochs = 20
# 
# # Learning Rate The Model Was Trained With
# lr = 0.0001
# 
# # Load model and tensorboard data to this folder
# folder = folder_name + str(batch_size) + "_lr" + str(lr)
# checkpoint_path = "./tensorboard/Models/" + folder + "/my_capsule_network"
# #learning_rate = learning_rate1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 70
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\3157251388.py", line 30, in <module>
    with open(dataJson, "r") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'data\\CatOrDog\\data.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most r

### Load the data from the test.h5 file

In [ ]:
def load_data(path):
    with h5py.File(path, 'r') as f:
        x = f['image'][()]
        y = f['label'][()]
    return np.array(x), np.array(y)


x_test, y_test = load_data(test_data)

# Normalize the image since we don't pass in normalized data but if the data is already normalized we can remove the / 255.
x_test = x_test.reshape((-1, input_dimensions[0], input_dimensions[1], input_dimensions[2])).astype('float32') / 255.
y_test = y_test.astype('float32')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 71
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\2653225637.py", line 8, in <module>
    x_test, y_test = load_data(test_data)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\2653225637.py", line 2, in load_data
    with h5py.File(path, 'r') as f:
NameError: name 'h5py' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

Du

## Loading the Model
The Cell Directly Below Is Template Code That Doesn't Need To Change

The Cell Below This One Can Be Modified To Use The Right Architecture

In [ ]:
def predictions(caps_Lminus1, dims_Lminus1, output_Lminus1,
                caps_L, dims_L, batch_size, weight_sharing=False, grid_cells_Lminus1=1, name=None):
    with tf.name_scope(name, default_name="predictions"):
        if weight_sharing == False:
            output_Lminus1 = tf.squeeze(output_Lminus1, axis=[1, 4], name="output_Lminus1")

        init_sigma = 0.1
        W_init = tf.random_normal(shape=(1, caps_Lminus1, caps_L, dims_L, dims_Lminus1),
                                  stddev=init_sigma, dtype=tf.float32, name="W_init")
        W = tf.Variable(W_init, name="W")
        tf.summary.histogram("W", W)

        W_tiled = tf.tile(W, [batch_size, grid_cells_Lminus1, 1, 1, 1], name="W_tiled")
        output_expanded_Lminus1 = tf.expand_dims(output_Lminus1, -1, name="output_expanded_Lminus1")
        output_tile_Lminus1 = tf.expand_dims(output_expanded_Lminus1, 2, name="output_tile_Lminus1")
        output_tiled_Lminus1 = tf.tile(output_tile_Lminus1, [1, 1, caps_L, 1, 1], name="output_tiled_Lminus1")
        predicted_L = tf.matmul(W_tiled, output_tiled_Lminus1, name="predicted_L")

        return predicted_L


def routing(caps_Lminus1, caps_L, batch_size, iterations, predicted_L, name=None):
    with tf.name_scope(name, default_name="routing"):
        raw_weights = tf.zeros([batch_size, caps_Lminus1, caps_L, 1, 1], dtype=np.float32, name="raw_weights")

        for r in range(0, iterations):
            routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")
            weighted_predictions = tf.multiply(routing_weights, predicted_L, name="weighted_predictions")
            weighted_sum = tf.reduce_sum(weighted_predictions, axis=(1), keepdims=True, name="weighted_sum")
            output_L = squash(weighted_sum, axis=-2, name="output_L")
            output_tiled_L = tf.tile(output_L, [1, caps_Lminus1, 1, 1, 1], name="output_tiled_L")
            agreement = tf.matmul(predicted_L, output_tiled_L, transpose_a=True, name="agreement")
            raw_weights = tf.add(raw_weights, agreement, name="raw_weights")

        return output_L, routing_weights


def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis, keepdims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector


def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis, keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)


X = tf.placeholder(shape=[None, input_dimensions[0], input_dimensions[1], input_dimensions[2]], dtype=tf.float32,
                   name="X")
tf.summary.image('X', X)
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 72
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\1079060265.py", line 53, in <module>
    X = tf.placeholder(shape=[None, input_dimensions[0], input_dimensions[1], input_dimensions[2]], dtype=tf.float32,
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception o

## Adjust This Cell As Needed Depending On The Architecture Used

In [ ]:
# Block 1
k = 3
s = 1
x = tf.layers.Conv2D(128, k, s, activation='relu')(X)
x = tf.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

# Block 2
k = 3
s = 1
x = tf.layers.Conv2D(256, k, s, activation='relu')(x)
x = tf.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

# Block 3
k = 7
s = 1
convcaps_dims = 16
convcaps_caps_types = 32
convcaps = tf.layers.Conv2D(convcaps_dims*convcaps_caps_types, k, s, name="convcaps")(x)
tf.summary.histogram("convcaps", convcaps)

convcaps_grid_length = tf.shape(convcaps)[1]
convcaps_grid_width = tf.shape(convcaps)[2]
convcaps_grid_cells = convcaps_grid_length * convcaps_grid_width
convcaps_caps = convcaps_caps_types * convcaps_grid_cells

convcaps_reshape = tf.reshape(convcaps, [-1, convcaps_caps, convcaps_dims], name="convcaps_reshape")
convcaps_output = squash(convcaps_reshape, name="convcaps_output")

batch_size = tf.shape(X)[0]

# If doing more than 1 capsule layer caps1_caps doesn't need to be the num_classes
caps1_caps = num_classes
caps1_dims = 12

caps1_predictions = predictions(convcaps_caps_types, convcaps_dims, convcaps_output,
                                caps1_caps, caps1_dims, batch_size, True, convcaps_grid_cells, "caps1_predictions")

caps1_output, routing1 = routing(convcaps_caps, caps1_caps,
                                 batch_size, 6, caps1_predictions, "routing1")

# You Can Add More Capsule Layers By Doing:
# Adding a second layer:
# -----------------------
# caps2_caps = 8
# caps2_dims = 16
# 
# caps2_predictions = predictions(caps1_caps, caps1_dims, caps1_output, 
#                                 caps2_caps, caps2_dims, batch_size, False, 1, "caps2_predictions")
#                                                                       ^ remains as False
# caps2_output, routing2 = routing(caps1_caps, caps2_caps, batch_size, 9, caps2_predictions, "routing2")
# 
# Adding a third layer:
# -----------------------
# caps3_caps = num_classes
# caps3_dims = 16
# 
# caps3_predictions = predictions(caps2_caps, caps2_dims, caps2_output, 
#                                 caps3_caps, caps3_dims, batch_size, False, 1, "caps3_predictions")
# 
# 
# caps3_output, routing3 = routing(caps2_caps, caps3_caps, batch_size, 9, caps3_predictions, "routing3")
# 
# Then Modify The Variable Below As Needed

# This Line Is Here To Make Using More Capsules Work Without Modifying Variables Below!
final_capsule_output, final_routing_output = caps1_output, routing1

# If more capsule Layers are introduced, the array below needs to be modified as well!
caps_and_routing = [convcaps_output, routing1, caps1_output]

# If Using More Capsules, The Above List Looks Like This:
# caps_and_routing = [convcaps_output, routing1, caps1_output, routing2, caps2_output, ...]

# Find out how many capsule Layers there are
capsule_layers = (len(caps_and_routing) - 1) / 2
print(f"{capsule_layers} Capsule Layer(s)")

# This can stay the same
caps_and_routing_dims = [convcaps_grid_length, convcaps_grid_width, convcaps_caps_types, num_classes]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 73
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\1373185723.py", line 4, in <module>
    x = tf.layers.Conv2D(128, k, s, activation='relu')(X)
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users

### Rest of the tempalte code

In [ ]:
with tf.name_scope(name="accuracy_cell"):
    lengths_temp = safe_norm(final_capsule_output, axis=-2, name="lengths_temp")
    lengths = tf.reshape(lengths_temp, shape=(-1, num_classes), name="lengths")
    lengths_argmax = tf.argmax(lengths, axis=1, name="lengths_argmax")
    y_predictions = lengths_argmax
    correct = tf.equal(y, y_predictions, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    tf.summary.scalar('accuracy', accuracy)

m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5

with tf.name_scope(name="loss_cell"):
    T = tf.one_hot(y, depth=caps1_caps, name="T")
    # Compute norm of each capsule in digitcaps
    caps1_output_norm = safe_norm(final_capsule_output, axis=-2, keep_dims=True, name="caps1_output_norm")

    present_error_raw = tf.square(tf.maximum(0., m_plus - caps1_output_norm), name="present_error_raw")
    present_error = tf.reshape(present_error_raw, shape=(-1, caps1_caps), name="present_error")
    # -1 tells reshape to calculate the size of this dimension.

    absent_error_raw = tf.square(tf.maximum(0., caps1_output_norm - m_minus), name="absent_error_raw")
    absent_error = tf.reshape(absent_error_raw, shape=(-1, caps1_caps), name="absent_error")
    # -1 tells reshape to calculate the size of this dimension.

    # Compute Margin Loss
    L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error, name="L")
    loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="loss")
    tf.summary.scalar('loss', loss)

# ### Adam Optimizer
# Initialize adam optimizer to minimize $\texttt{loss}$.
with tf.name_scope(name="train"):
    learning_rate = tf.placeholder(tf.float32, shape=[])
    tf.summary.scalar('learning_rate', learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss, name="training_op")

merged_summary = tf.summary.merge_all()
init = tf.global_variables_initializer()
saver = tf.train.Saver()


def visualize_convcaps(caps_and_routing, caps_and_routing_dims, reduce):
    temp_dims = caps_and_routing_dims[0:3] + [1] * (len(caps_and_routing_dims) - 3)
    convcaps_lengths = tf.reshape(safe_norm(caps_and_routing[0], axis=-1), temp_dims)
    temp_dims = [1] * (3) + caps_and_routing_dims[3:len(caps_and_routing_dims)]
    convcaps_lengths_tiled = tf.tile(convcaps_lengths, temp_dims)

    temp_dims = caps_and_routing_dims[0:4] + [1] * (len(caps_and_routing_dims) - 4)
    routing1_reshape = tf.reshape(caps_and_routing[1], temp_dims)
    temp_dims = [1] * (4) + caps_and_routing_dims[4:len(caps_and_routing_dims)]
    routing1_reshape_tiled = tf.tile(routing1_reshape, temp_dims)

    temp_dims = [1, 1, 1, caps_and_routing_dims[3]] + [1] * (len(caps_and_routing_dims) - 4)
    caps1_lengths = tf.reshape(safe_norm(caps_and_routing[2], axis=-2), temp_dims)
    temp_dims = caps_and_routing_dims[0:3] + [1] + caps_and_routing_dims[4:len(caps_and_routing_dims)]
    caps1_lengths_tiled = tf.tile(caps1_lengths, temp_dims)

    temp1 = tf.square(tf.multiply(routing1_reshape_tiled, caps1_lengths_tiled))

    temp_matrices = []
    index = 0
    for i in range(4, len(caps_and_routing_dims)):
        temp_dims = [1] * (i - 1) + caps_and_routing_dims[i - 1:i + 1] + [1] * (len(caps_and_routing_dims) - i - 1)
        routing_reshape = tf.reshape(caps_and_routing[3 + index], temp_dims)
        temp_dims = caps_and_routing_dims[0:i - 1] + [1] * (2) + caps_and_routing_dims[
                                                                 i + 1:len(caps_and_routing_dims)]
        routing_reshape_tiled = tf.tile(routing_reshape, temp_dims)

        temp_dims = [1] * len(caps_and_routing_dims)
        temp_dims[i] = caps_and_routing_dims[i]
        caps_lengths = tf.reshape(safe_norm(caps_and_routing[3 + index + 1], axis=-2), temp_dims)
        temp_dims = caps_and_routing_dims.copy()
        temp_dims[i] = 1
        caps_lengths_tiled = tf.tile(caps_lengths, temp_dims)

        temp = tf.square(tf.multiply(routing_reshape_tiled, caps_lengths_tiled))
        temp_matrices.append(temp)

        index = index + 2

    all_paths = tf.multiply(convcaps_lengths_tiled, temp1)
    for temp_matrice in temp_matrices:
        all_paths = tf.multiply(all_paths, temp_matrice)

    normalizing_factor = np.prod(caps_and_routing_dims[2:len(caps_and_routing_dims)])
    all_paths_average = tf.reduce_sum(all_paths, axis=reduce) / normalizing_factor

    return all_paths_average

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 74
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\839322430.py", line 1, in <module>
    with tf.name_scope(name="accuracy_cell"):
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desk

Testtest

In [ ]:
# What colourmap to use
cmap = "turbo"

# How Strong The Overlayed Image Is
alpha = 0.5

# Index To Start At
startIndex_ = 16

# Index To End At
endIndex_ = 32

# Number Of Columns To Display, Rows Are Calculated Automatically
num_cols = 4

# Specify Which Axis To Reduce
# For Simplicity This Views All Capsule Layers But Can Be Modified Like So:
# reduce = [2] <-- only first capsule layer
# reduce = [3] <-- only second capsule layer
# reduce = [2,3] <-- only first and second capsule layer
# reduce = [3,4] <-- only second and third capsule layer
reduce = list(range(2, 2 + int(capsule_layers)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 75
Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\1232991048.py", line 22, in <module>
    reduce = list(range(2, 2 + int(capsule_layers)))
NameError: name 'capsule_layers' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_4328\1232991048.py", line 22, in <module>
    reduce = list(range(2, 2 + int(capsule_layers)))
NameError: name 'capsule_layers' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Michael\Desktop\Capsule_Networks_Rpv _Final\.venv\lib\site-packages\IPython\core\inter

In [1]:
results = []

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)

    for i in range(startIndex_, endIndex_):
        # Run the session to get outputs
        out = sess.run([visualize_convcaps(caps_and_routing, caps_and_routing_dims, reduce), y_predictions, lengths],
                       feed_dict={
                           X: x_test[i].reshape([-1, input_dimensions[0], input_dimensions[1], input_dimensions[2]]),
                           y: [y_test[i]]
                       })
        
        feature_map = out[0]  # Keep the (x, y, num_class) shape
        prediction = out[1][0]  # Predicted value
        ground_truth = y_test[i]  # Actual label
        confidenceScore = out[2].flatten().tolist()  # Confidence scores
        results.append((feature_map, prediction, ground_truth, confidenceScore, i))

# Number of images to plot
num_images = endIndex_ - startIndex_

# Determine the grid size for subplots (e.g., 2x2 grid for 4 images)
num_rows = (num_images + num_cols - 1) // num_cols

# Create a figure with the desired number of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 10), facecolor="black")

# Flatten axes array for easy indexing
axes = axes.flatten()

for idx, result in enumerate(results):
    feature_map, prediction, ground_truth, confidenceScore, image_index = result

    base_image = x_test[image_index]

    # Resize the feature map to match the base image dimensions
    scale_factor = (input_dimensions[0] / feature_map.shape[0], input_dimensions[1] / feature_map.shape[1])
    resized_feature_map = zoom(feature_map[:, :, int(ground_truth)], scale_factor)

    # Plot the base image in the corresponding subplot
    axes[idx].imshow(base_image, cmap='gray')

    # Overlay the resized feature map with some transparency
    axes[idx].imshow(resized_feature_map, cmap=, alpha=0.5)

    # Set title for each subplot
    axes[idx].set_title(f'Expected: {class_names[int(ground_truth)]} | Got: {class_names[int(prediction)]}', color="white")
    axes[idx].axis('off')

# Hide any unused subplots
for j in range(num_images, num_rows * num_cols):
    axes[j].axis('off')

plt.tight_layout()

# Display all plots
plt.show()

SyntaxError: invalid syntax (1225425983.py, line 45)

ERROR! Session/line number was not unique in database. History logging moved to new session 76
